# 闯关题3

## STEP1: 导入数据，查看数据  
导入数据  

1. 问题q1：查看数据集有哪些列，并存入变量a1  
2. 问题q2：查看反应时(“rt”)这一列中数值的类型，并存入变量a2  
3. 问题q3：统计反应时(“rt”)这一列包含缺失值的个数，并存入变量a3  
4. 问题q4：统计反应时大于300且小于1000的行数，并存入变量a4

In [1]:
import pandas as pd

# 从名为 "stroop.csv" 的 CSV 文件中读取数据并存储在 DataFrame 中
df = pd.read_csv("/home/mw/input/84988498/stroop.csv")

# 显示 DataFrame 的前几行数据
df.head()

# 获取 DataFrame 的列名
a1 = df.columns

# 获取 "rt" 列的数据类型
a2 = df["rt"].dtype

# 统计 "rt" 列中缺失值的数量
a3 = df["rt"].isnull().sum()

# 统计 "rt" 列中数值在 300 到 1000 之间的数据数量
a4 = ((df["rt"] >= 300) & (df["rt"] <= 1000)).sum()


## STEP2：先对被试的反应时进行处理  
1. 删除所有的练习试次  
2. 删除未进行反应的试次（被试未反应的试次中，rt被赋值为-1）  
3. 删除反应错误的试次（被试错误反应的试次中，correct被赋值为0）  
4. 按照被试、条件进行分组后，计算rt这一列的平均值  
5. 将获得的结果存在一个新的数据框(df_rt_mean)内

In [2]:
# 根据条件筛选数据：实验阶段不为 "practice"，rt大于等于 0，且正确率为 1 的数据
df_rt_mean = df[(df["exp_stage"] != "practice") & (df["rt"] >= 0) & (df["correct"] == 1)]

# 按照 "worker_id" 和 "condition" 分组，并计算每组中 "rt" 列的平均值
df_rt_mean = df_rt_mean.groupby(["worker_id", "condition"])["rt"].mean()

# 重置索引，将分组结果转换为 DataFrame
df_rt_mean = df_rt_mean.reset_index()


## STEP3：对被试的正确率进行处理  
1. 删除所有的练习试次（注意：相比于反应时，此处不需要删除被试未反应和错误反应的试次）  
2. 按照被试、条件进行分组后，计算correct这一列的平均值  
5. 将获得的结果存在一个新的数据框(df_acc)内

In [3]:
# 根据条件筛选数据：实验阶段不为 "practice" 的数据
df_acc = df[df["exp_stage"] != "practice"]

# 按照 "worker_id" 和 "condition" 分组，并计算每组中 "correct" 列的平均值
df_acc = df_acc.groupby(["worker_id", "condition"])["correct"].mean()

# 重置索引
df_acc = df_acc.reset_index()


## STEP4：合并两个数据框 df_rt_mean, df_acc

In [4]:
# 将 df_rt_mean 和 df_acc 进行拼接，inner表示按照两个数据框共有的列进行拼接
df_rt_acc = pd.merge(df_rt_mean, df_acc, how="inner")

## STEP5：绘制反应时的箱线图与密度分布图  

1.绘制出一致条件与不一致条件下的反应时分布密度图(density plot)  

2.分别绘制出一致条件与不一致条件下的反应时箱线图(boxplot)  


根据图像内容回答以下问题，将选择题答案填入下方代码cell中对应的变量内。  

> **问题Q5: 从图像上看，一致条件下的反应时分布更接近以下哪种分布  
>A.左偏分布  
>B.右偏分布  
>C.近似正态分布**  
>*(注意：如果不了解偏态分布定义，请先自行搜索)*  


> **问题Q6：从图像上看，不一致条件下的反应时分布的中位数在哪个范围内  
> A.600-700  
> B.700-800  
> C.800-900**

In [5]:
import seaborn as sns

sns.displot(df_rt_acc, x = 'rt', hue = 'condition', kind='kde')

<Figure size 633.625x500 with 1 Axes>

In [6]:
sns.boxplot(df_rt_acc, x = 'condition', y = 'rt')

<Axes: xlabel='condition', ylabel='rt'>

<Figure size 640x480 with 1 Axes>

In [7]:
a5 = 'B'  # 在''中间填写你的 Q5 选项，实现效果如 a5 = 'A'
a6 = 'B'  # 在''中间填写你的 Q6 选项

## STEP6：将df_rt_acc由长数据转化为宽数据   

1. 将长数据转化为宽数据  
2. 对最后生成的列名进行重命名，分别是  
>* 被试编号：'worker_id',  
>* 一致条件下的反应时均值：'rt_congruent',  
>* 不一致条件下的反应时均值：'rt_incongruent',  
>* 一致条件下的反应时正确率：'acc_congruent',  
>* 不一致条件下的反应时正确率：'acc_incongruent'  

3. 最后的数据框仍存为“df_rt_acc”  
4. 选取'acc_incongruent'列 的最后10行，存入变量**a7**

In [8]:
# 将 "worker_id" 和 "condition" 设置为索引列，方便后续使用unstack
df_rt_acc = df_rt_acc.set_index(keys=['worker_id', 'condition'])

# 使用unstack将长数据转化为宽数据
# 在unstack之后，"worker_id" 和 "condition" 列被放置在了多层索引的最外层。
# 使用了 reset_index()，将这两列还原为 DataFrame 的普通列，同时重新生成默认的整数索引
df_rt_acc = df_rt_acc.unstack().reset_index()

# 重新命名列名，更具体地表示数据含义
df_rt_acc.columns = ['worker_id',
                     'rt_congruent',
                     'rt_incongruent',
                     'acc_congruent',
                     'acc_incongruent']

#保留两位小数
df_rt_acc = df_rt_acc.round(2)

In [9]:
df_rt_acc

,worker_id,rt_congruent,rt_incongruent,acc_congruent,acc_incongruent
0,s001,692.05,821.67,0.88,0.81
1,s002,680.51,797.26,0.98,0.98
2,s003,578.54,667.33,1.00,0.96
3,s004,720.35,805.56,1.00,1.00
4,s005,555.31,596.48,1.00,1.00
...,...,...,...,...,...
517,s554,621.15,838.48,0.98,0.88
518,s556,887.62,1062.39,0.94,0.96
519,s557,650.26,770.54,0.96,0.77
520,s559,677.36,876.78,0.98,0.85


In [10]:
a7 = df_rt_acc["acc_incongruent"].iloc[-10:]
a7

512    0.96
513    1.00
514    0.96
515    0.77
516    0.94
517    0.88
518    0.96
519    0.77
520    0.85
521    0.98
Name: acc_incongruent, dtype: float64

## STEP7：将结果保存为 csv 文件  
💡 这一步的代码你无需修改，直接运行即可。

In [11]:
answer = pd.DataFrame({'id':[f"q{i}" for i in range(1,8)],
                        'answer':[a1.to_list(),
                                    a2,
                                    a3,
                                    a4,
                                    a5,
                                    a6,
                                    a7.to_list()]})

answer.to_csv("answer_3.csv", index=None)

In [12]:
answer

,id,answer
0,q1,"[Unnamed: 0, battery_name, condition, correct,..."
1,q2,int64
2,q3,0
3,q4,53721
4,q5,B
5,q6,B
6,q7,"[0.96, 1.0, 0.96, 0.77, 0.94, 0.88, 0.96, 0.77..."
